# **数据规整：聚合、合并和重塑**
---

## **层次化索引**
---

In [ ]:
import pandas as pd
import numpy as np
data = pd.Series(np.random.randn(9),
                        index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]]
                        )

data

> **内层都可以索引**

In [ ]:
data.loc[['b', 'd'],3]

> **data.unstack()逆透视**

In [ ]:
data.unstack()

> **data.stack()透视**

In [ ]:
data.unstack().stack()

> **DataFrame每条轴都可以有分层索引**

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'],
                    ['Green', 'Red', 'Green']]
                    )
frame

In [ ]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
# frame.index
# frame.columns
# 注意索引名、列名、行名、字段名/列名/列标签是不一样的东西
frame

In [ ]:
frame['Ohio']

### **重排与分级排序**

> **df.swaplevel(x,y)交换排序位置**

In [ ]:
frame.swaplevel('key1', 'key2')

In [ ]:
frame.sort_index(level=1)

> **sort_index(level=0级别）**

In [ ]:
frame.swaplevel(0, 1).sort_index(level=0)

### **根据级别汇总统计**

In [ ]:
frame.sum(level='key2')

In [ ]:
frame

In [ ]:
frame.sum(level='color', axis=1)

### **使用DataFrame的列进行索引**

> **set_index()**

In [ ]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                     'c': ['one', 'one', 'one', 'two', 'two',
                     'two', 'two'],
                     'd': [0, 1, 2, 0, 1, 2, 3]})
frame

In [ ]:
frame2 = frame.set_index(['c','d'],drop=False) # 保留相关列
frame2

## **合并数据集**  
---
> - pandas.merge可根据一个或多个键将不同DataFrame中的行连接起来。SQL或其他关系型数据库的用户对此应该会比较熟悉，因为它实现的就是数据库的join操作。  
> - pandas.concat可以沿着一条轴将多个对象堆叠到一起。  
> - 实例方法combine_first可以将重复数据拼接在一起，用一个对象中的值填充另一个对象中的缺失值。

### **类似数据库的合并**

> **pd.merge(df1,df2,on='key')**

In [33]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                     'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [35]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [36]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [37]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [39]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                     'data1': range(7)})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [40]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                        'data2': range(3)})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [41]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [42]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


> **连接方式**  
> **inner ：内连接**  
> **left : 左连接**  
> **right :右连接**  
> **Outer : 外连接**

In [44]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [46]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                     'data2': range(5)})
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [47]:
pd.merge(df1,df2,on='key',how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


> **多键合并on=['key1', 'key2']传入一个由列名组成的列表即可**

In [48]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                    'key2': ['one', 'two', 'one'],
                    'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                         'key2': ['one', 'one', 'one', 'two'],
                          'rval': [4, 5, 6, 7]})

pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [49]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


> **suffixes('_left', '_right')指定重复项的列明**

In [50]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


> merge参数
> ![](img/merge%E5%8F%82%E6%95%B0.jpg)

### **索引上的合并**

In [52]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                     'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
pd.merge(left1, right1, left_on='key', right_index=True) # 右边使用索引作为合并的字段

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


> **merge默认取交集，可加how='outer'取并集 **

In [53]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


> **多层次的索引，多键合并**

In [54]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                                'Nevada', 'Nevada'],
                        'key2': [2000, 2001, 2002, 2001, 2002],
                         'data': np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                        index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                               'Ohio', 'Ohio'],
                               [2001, 2000, 2000, 2000, 2001, 2002]],
                       columns=['event1', 'event2'])

pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [55]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
          right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


> **直接使用索引轴合并**

In [58]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                        index=['a', 'c', 'e'],
                       columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                     index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### **Join方法**

In [59]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### **concat()轴向连接**

In [60]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [61]:
pd.concat([s1,s2,s3],axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [65]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [63]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


> **Join=‘inner’取交集**

In [64]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [ ]:
pd.concat([s1, s4], axis=1, join_axes=['a', 'c', 'b', 'e'])

> **合并中还可以创建层次化索引，使用keys**

In [68]:
result = pd.concat([s1, s1, s3], keys=['one','two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [70]:
result.unstack() # 逆透视一下

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [71]:
pd.concat([s1, s2, s3], axis=1, keys=['one','two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


### **同样的逻辑也适用于DataFrame对象**

In [72]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                    columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                     columns=['three', 'four'])
df1                     

,one,two
a,0,1
b,2,3
c,4,5


In [73]:
df2

,three,four
a,5,6
c,7,8


In [75]:
pd.concat([df1,df2],axis=1,keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [77]:
# 如果传入的值的字典，那字典的键会被当作keys
pd.concat({'level1': df1, 'level2': df2}, axis=1) 

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [78]:
# 设置索引层次水平
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
       names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

> ![](img/concat%E5%8F%82%E6%95%B0.jpg)

### **合并重叠数据combine_first打补丁**

In [79]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
               index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [82]:
import numpy as np
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [83]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

## **重塑和轴向旋转**
---

### **重塑层次化索引**
> **df.stack()透视和df.unstack()逆透视**

In [84]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                      index=pd.Index(['Ohio','Colorado'], name='state'),
                     columns=pd.Index(['one', 'two', 'three'],
                       name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [86]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [87]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


###  **将“长格式”旋转为“宽格式”**

In [88]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [89]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')

columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

data = data.reindex(columns=columns)

data.index = periods.to_timestamp('D', 'end')

ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [90]:
data # date和item通常就是主键

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [92]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [94]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]


,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.664368
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.472860
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.442369
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.006943
4,1959-06-30 23:59:59.999999999,infl,2.340,1.437116
5,1959-06-30 23:59:59.999999999,unemp,5.100,1.116023
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.891166
7,1959-09-30 23:59:59.999999999,infl,2.740,-1.004740
8,1959-09-30 23:59:59.999999999,unemp,5.300,-0.515419
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,1.172333


In [95]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.472860  0.664368   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.437116  0.006943   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -1.004740  0.891166   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.544089  1.172333   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.793611  0.495261   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.442369  
1959-06-30 23:59:59.999999999  1.116023  
1959-09-30 23:59:59.999999999 -0.515419  
1959-12-31 23:59:59.999999999 -0.464534  
1960-03-31 23:59:59.999999999 -0.339214

> **_注意，pivot其实就是用set_index创建层次化索引，再用unstack重塑：_**

### **将“宽格式”旋转为“长格式”**

In [96]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                       'A': [1, 2, 3],
                       'B': [4, 5, 6],
                      'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [97]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [98]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [ ]:
reshaped.reset_index() 